In [33]:
import pandas as pd
import numpy as np
import random
import os
import joblib
from scipy.stats import rankdata
import warnings
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import miceforest as mf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')

In [ ]:
class Config:
    SEED = 42
    N_SPLITS = 10
    N_TOP_FOLDS = 5

    TRAIN_PATH = './data/train.csv'
    TEST_PATH = './data/test.csv'
    SUBMISSION_PATH = './data/sample_submission.csv'
    MICE_MODEL_PATH = './kds_model.pkl'
    OUTPUT_FILENAME = 'final_ensemble_submission.csv'

    LGB_PARAMS = {
        'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
        'n_estimators': 2000, 'learning_rate': 0.01, 'num_leaves': 20,
        'max_depth': 5, 'seed': SEED, 'n_jobs': -1, 'verbose': -1,
        'colsample_bytree': 0.7, 'subsample': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 0.1
    }
    XGB_PARAMS = {
        'objective': 'binary:logistic', 'eval_metric': 'auc',
        'n_estimators': 2000, 'learning_rate': 0.01, 'max_depth': 5,
        'seed': SEED, 'n_jobs': -1, 'colsample_bytree': 0.8, 'subsample': 0.8, 'early_stopping_rounds': 200
    }
    CAT_PARAMS = {
        'loss_function': 'Logloss', 'eval_metric': 'AUC', 'iterations': 3000,
        'learning_rate': 0.05, 'depth': 7, 'random_seed': SEED,
        'verbose': 0, 'task_type': 'CPU',
        'bagging_temperature': 0.01, 'l2_leaf_reg': 5, 'random_strength': 0.9,
        'auto_class_weights': 'Balanced', 'early_stopping_rounds': 200
    }
    ENSEMBLE_WEIGHTS = {'lgbm': 1.0, 'xgb': 1.2, 'cat': 1.2}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

In [35]:
class DataManager:
    @staticmethod
    def load_data():
        train_df = pd.read_csv(Config.TRAIN_PATH)
        test_df = pd.read_csv(Config.TEST_PATH)
        test_ids = test_df['ID']
        train_df.drop(columns=['ID'], inplace=True)
        test_df.drop(columns=['ID'], inplace=True)
        return train_df, test_df, test_ids

In [ ]:
class FeatureEngineer:
    def __init__(self, seed):
        self.seed = seed

    def _number_mapping(self, df):
        ivf_cols = ['IVF 임신 횟수', 'IVF 출산 횟수', 'DI 임신 횟수', 'DI 출산 횟수', 'IVF 시술 횟수', '총 임신 횟수', 'DI 시술 횟수', '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 출산 횟수']
        mapping = {'0회': 0, '1회': 1, '2회': 2, '3회': 3, '4회': 4, '5회': 5, '6회 이상': 6}
        for col in ivf_cols:
            if col in df.columns: df[col] = pd.to_numeric(df[col].replace(mapping), errors='coerce')
        return df

    def _handle_di_missing(self, df):
        di_nan_col = ['미세주입된 난자 수', '미세주입에서 생성된 배아 수', '총 생성 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 채취 경과일', '난자 해동 경과일', '배아 이식 경과일', '배아 해동 경과일', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '단일 배아 이식 여부', '난자 혼합 경과일', '임신 시도 또는 마지막 임신 경과 연수']
        row = df['시술 유형'] == "DI"
        df.loc[row, di_nan_col] = df.loc[row, di_nan_col].fillna(0)
        return df

    def _get_or_train_mice_model(self, df, target_columns, model_path):
        try:
            mice_mdl = joblib.load(model_path)
            print("Pre-trained MICE model loaded successfully.")
        except FileNotFoundError:
            print("MICE model not found. Training a new MICE model...")
            df_subset = df[target_columns].copy()
            mice_mdl = mf.ImputationKernel(df_subset, random_state=self.seed)
            mice_mdl.mice(iterations=5, n_estimators=50, n_jobs=-1)
            joblib.dump(mice_mdl, model_path)
            print(f"New MICE model trained and saved to {model_path}")
        return mice_mdl

    def _impute_with_mice(self, train_df, test_df, model_path):
        target_cols = ['미세주입된 난자 수', '미세주입에서 생성된 배아 수', '총 생성 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 채취 경과일', '난자 해동 경과일', '배아 이식 경과일', '배아 해동 경과일', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '단일 배아 이식 여부', '난자 혼합 경과일', '임신 시도 또는 마지막 임신 경과 연수']
        mice_mdl = self._get_or_train_mice_model(train_df, target_cols, model_path)
        
        train_imputed = train_df.copy()
        train_imputed[target_cols] = mice_mdl.complete_data()
        print('MICE: Training data imputed.')

        test_imputed = test_df.copy()
        test_kernel = mice_mdl.impute_new_data(test_df[target_cols])
        test_imputed[target_cols] = test_kernel.complete_data()
        print('MICE: Test data imputed.')
        return train_imputed, test_imputed

    def _create_features(self, df):
        df['난자 성공률'] = np.where(df['수집된 신선 난자 수'] == 0, -1, df['미세주입된 난자 수'] / df['수집된 신선 난자 수'])
        df['저장된 배아 사용률'] = np.where(df['총 생성 배아 수'] + df['해동된 배아 수'] == 0, -1, df['저장된 배아 수'] / (df['총 생성 배아 수'] + df['해동된 배아 수']))
        df['IVF 임신률'] = np.where(df['IVF 시술 횟수'] == 0, -1, df['IVF 임신 횟수'] / df['IVF 시술 횟수'])
        df['총 출산률'] = np.where(df['총 임신 횟수'] == 0, -1, df['총 출산 횟수'] / df['총 임신 횟수'])
        df['미세주입 배아 생성률'] = np.where(df['미세주입된 난자 수'] == 0, -1, df['미세주입에서 생성된 배아 수'] / df['미세주입된 난자 수'])
        df['총 사용 배아'] = df['해동된 배아 수'] + df['총 생성 배아 수']
        df['대리모 여부'] = df['대리모 여부'].fillna(-1)
        df['시술 당시 나이'] = df['시술 당시 나이'].replace('알 수 없음', '만45-50세')
        df["난자 기증자 나이"] = df["난자 기증자 나이"].replace({'알 수 없음': '만 21-25세'})
        pg_cols = ["착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "PGD 시술 여부", "PGS 시술 여부"]
        for col in pg_cols: df[col] = df[col].fillna(0)
        df["과거 유전자 검사 사용 여부"] = df["착상 전 유전 검사 사용 여부"] + df["착상 전 유전 진단 사용 여부"]
        df["현재 검사 사용 여부"] = df["PGD 시술 여부"] + df["PGS 시술 여부"]
        df["나이"] = df["시술 당시 나이"]
        mask_donor = (df["난자 출처"] == "기증 제공") & (df["난자 기증자 나이"] != "알 수 없음")
        df.loc[mask_donor, "나이"] = df.loc[mask_donor, "난자 기증자 나이"]
        male_fail = ["불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]
        female_fail = ["불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증"]
        df["남성 불임 심각도"] = df[male_fail].sum(axis=1)
        df["여성 불임 심각도"] = df[female_fail].sum(axis=1)
        df['난임 여부'] = df['총 시술 횟수'] - df['총 임신 횟수']
        df['유산 여부'] = (df['총 임신 횟수'] - df['총 출산 횟수']).apply(lambda x: 1 if x > 0 else 0)
        return df

    def transform(self, train_df, test_df):
        train_df = self._handle_di_missing(train_df)
        test_df = self._handle_di_missing(test_df)
        train_df, test_df = self._impute_with_mice(train_df, test_df, Config.MICE_MODEL_PATH)
        train_df = self._number_mapping(train_df)
        test_df = self._number_mapping(test_df)
        train_df = self._create_features(train_df)
        test_df = self._create_features(test_df)
        y_train = train_df['임신 성공 여부']
        drop_cols = ['임신 성공 여부', '불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인']
        train_df.drop(columns=drop_cols, inplace=True, errors='ignore')
        test_df.drop(columns=drop_cols, inplace=True, errors='ignore')

        categorical_features = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in categorical_features:
            train_df[col] = train_df[col].astype(str).astype('category')
            test_df[col] = test_df[col].astype(str).astype('category')

        return train_df, test_df, y_train, categorical_features

In [ ]:
class EnsembleTrainer:
    def __init__(self, lgb_params, xgb_params, cat_params, n_splits, seed):
        self.lgb_params = lgb_params
        self.xgb_params = xgb_params
        self.cat_params = cat_params
        self.skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    def train(self, X, y, categorical_features):
        auc_scores = {'lgbm': [], 'xgb': [], 'cat': []}
        test_preds = {'lgbm': [], 'xgb': [], 'cat': []}

        scale_pos_weight = y.value_counts()[0] / y.value_counts()[1]
        self.lgb_params['scale_pos_weight'] = scale_pos_weight
        self.xgb_params['scale_pos_weight'] = scale_pos_weight

        for fold, (train_idx, valid_idx) in enumerate(self.skf.split(X, y)):
            print(f'===== Fold {fold+1}/{self.skf.get_n_splits()} =====')
            X_tr, X_val = X.iloc[train_idx], X.iloc[valid_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[valid_idx]

            self._train_lgbm(X_tr, y_tr, X_val, y_val, auc_scores, test_preds, X, fold)
            self._train_xgb(X_tr, y_tr, X_val, y_val, auc_scores, test_preds, X, categorical_features, fold)
            self._train_cat(X_tr, y_tr, X_val, y_val, auc_scores, test_preds, X, categorical_features, fold)

        print("All models training complete.")
        return auc_scores, test_preds

    def _train_lgbm(self, X_tr, y_tr, X_val, y_val, scores, preds, X_test, fold):
        print("Training LightGBM...")
        model = lgb.LGBMClassifier(**self.lgb_params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], eval_metric='auc', callbacks=[lgb.early_stopping(100, verbose=False)])
        pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, pred)
        scores['lgbm'].append((auc, fold))
        preds['lgbm'].append(model.predict_proba(X_test)[:, 1])
        print(f"LGBM AUC: {auc:.5f}")

    def _train_xgb(self, X_tr, y_tr, X_val, y_val, scores, preds, X_test, cat_feats, fold):
        print("Training XGBoost...")
        X_tr_xgb, X_val_xgb, X_test_xgb = X_tr.copy(), X_val.copy(), X_test.copy()
        for col in cat_feats:
            X_tr_xgb[col] = X_tr_xgb[col].cat.codes
            X_val_xgb[col] = X_val_xgb[col].cat.codes
            X_test_xgb[col] = X_test_xgb[col].cat.codes
        model = xgb.XGBClassifier(**self.xgb_params)
        model.fit(X_tr_xgb, y_tr, eval_set=[(X_val_xgb, y_val)], verbose=False)
        pred = model.predict_proba(X_val_xgb)[:, 1]
        auc = roc_auc_score(y_val, pred)
        scores['xgb'].append((auc, fold))
        preds['xgb'].append(model.predict_proba(X_test_xgb)[:, 1])
        print(f"XGBoost AUC: {auc:.5f}")

    def _train_cat(self, X_tr, y_tr, X_val, y_val, scores, preds, X_test, cat_feats, fold):
        print("Training CatBoost...")
        model = CatBoostClassifier(**self.cat_params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], cat_features=cat_feats, verbose=0)
        pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, pred)
        scores['cat'].append((auc, fold))
        preds['cat'].append(model.predict_proba(X_test)[:, 1])
        print(f"CatBoost AUC: {auc:.5f}")

In [38]:
def generate_submission(test_preds, auc_scores, weights, top_n, test_ids, filename):
    def weighted_rank_ensemble(predictions, weights):
        ranked_preds = np.array([rankdata(pred) for pred in predictions])
        weighted_avg_rank = np.average(ranked_preds, axis=0, weights=weights)
        return weighted_avg_rank / np.max(weighted_avg_rank)

    selected_preds = []
    selected_weights = []

    print(f"Ensembling top {top_n} folds from each model...")
    for model_type in ['lgbm', 'xgb', 'cat']:
        top_folds = sorted(auc_scores[model_type], reverse=True, key=lambda x: x[0])[:top_n]
        fold_indices = [fold_idx for _, fold_idx in top_folds]
        print(f"{model_type.upper()} selected folds (AUC): {[(f'{auc:.5f}', idx) for auc, idx in top_folds]}")
        selected_preds += [test_preds[model_type][idx] for idx in fold_indices]
        selected_weights += [weights[model_type]] * len(fold_indices)

    final_predictions = weighted_rank_ensemble(selected_preds, selected_weights)
    submission_df = pd.DataFrame({'ID': test_ids, 'probability': final_predictions})
    submission_df.to_csv(filename, index=False)
    print(f"Submission file '{filename}' created successfully!")

In [ ]:
if __name__ == '__main__':
    # Load Data
    train_raw, test_raw, test_ids = DataManager.load_data()

    # Feature Engineering
    feature_engineer = FeatureEngineer(seed=Config.SEED)
    X_train, X_test, y_train, cat_features = feature_engineer.transform(train_raw, test_raw)

    # Train Ensemble Models
    trainer = EnsembleTrainer(Config.LGB_PARAMS, Config.XGB_PARAMS, Config.CAT_PARAMS, Config.N_SPLITS, Config.SEED)
    auc_scores_dict, test_pred_dict = trainer.train(X_train, y_train, cat_features)

    # Generate Submission File
    generate_submission(test_pred_dict, auc_scores_dict, Config.ENSEMBLE_WEIGHTS, Config.N_TOP_FOLDS, test_ids, Config.OUTPUT_FILENAME)